# Import

In [56]:
from ray.rllib.examples.policy.rock_paper_scissors_dummies import (
    BeatLastHeuristic,
    AlwaysSameHeuristic,
)
import os
import random
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from gym.spaces import Tuple, Discrete, Box,Dict
import ray
from ray import air
from ray import tune
from ray.tune.registry import register_env
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import numpy as np
from ray.rllib.utils import check_env
from typing import Optional
from ray.rllib.algorithms.pg import (
    PG,
    PGConfig,
    PGTF2Policy,
    PGTF1Policy,
    PGTorchPolicy,
)
import argparse
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.algorithms.maddpg.maddpg import MADDPGConfig,DQN

In [57]:
class TestEnv(MultiAgentEnv):
    def __init__(self, config):
        super().__init__()
        # Mosse 
        self.MOVES = ['mossa1', 'mossa2','mossa3']
        # Questa è la truncation cosi esce per non girare all'infinito
        self.NUM_ITERS = 20
        # Mappa che in base all'azione eseguita mi da costo, impatto, ecc dell'azione
        self.REWARD_MAP = {
            'mossa1': (1, 0, 0),
            'mossa2': (2, 0, 0),
            'mossa3': (3, 0, 0)
        }
        # per la funzione di reward
        self.wt = 0.5
        self.wc = 0.5
        self.wi = 0.5
        self.tMax = 100
        self.cMax = 100
        self.possible_agents = ['attaccante','difensore']
        self._agent_ids = set(self.possible_agents)
        self._action_spaces = {agent: Discrete(3) for agent in self.possible_agents}
        self._observation_spaces = {
            agent:Dict(
                {
                    'observation': Box(low=0,high=1,shape=(2,),dtype=np.float32),
                }
            )
            for agent in self.possible_agents
        }
        

    def step(self,allAction):
        """ if (
            self.terminations[self.agent_selection]
            or self.truncations[self.agent_selection]
        ):
            #self._was_dead_step(action)
            print('ECCOOOO:',self.agents)
            return """

        for agent in self.agents:
            #agent = self.agent_selection
            print('Agente in azione:',agent)
            """ action = allAction[agent]
            reward = self.REWARD_MAP[self.MOVES[action]]
            valReward = -self.wt*(reward[0]/self.tMax)-self.wc*(reward[1]/self.cMax)-self.wi*(1) 
            #rewardInv = -valReward
            print('Reward:',valReward)
            if agent == 'attaccante':
                self.rewards[self.agents[0]], self.rewards[self.agents[1]] = (valReward,0)
            else:
                self.rewards[self.agents[0]], self.rewards[self.agents[1]] = (0,valReward) """
            self.num_moves += 1
            # The truncations dictionary must be updated for all players.
            self.truncations = {
                agent: self.num_moves >= self.NUM_ITERS for agent in self.agents
            }
            #self.agent_selection = self._agent_selector.next()
            # Adds .rewards to ._cumulative_rewards
            #self._accumulate_rewards() 
        return self._observation_spaces,self.rewards,self.terminations,self.truncations,self.infos
        

        
    def reset(self,seed=None,options=None):
        super().reset()
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}      
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        #self.state = {agent: NONE for agent in self.agents}
        #self.observations = {agent: NONE for agent in self.agents}
        # credo serve per arrestare
        self.num_moves = 0
        
        #Our agent_selector utility allows easy cyclic stepping through the agents list.
        
        #self._agent_selector = agent_selector(self.agents)
        #self.agent_selection = self._agent_selector.reset()
        return self._observation_spaces,self.infos
    
    def observation_space(self, agent):
        # gymnasium spaces are defined and documented here: https://gymnasium.farama.org/api/spaces/
        return self._observation_spaces[agent]
    
    def action_space(self, agent):
        return self._action_spaces[agent]
        


def env_creator(args):
    env = TestEnv(args)

    obs_space = {agent: env.observation_space(agent)['observation'] for agent in env.possible_agents}
    act_space = {agent: env.action_space(agent) for agent in env.possible_agents}
    print('obs_space:',obs_space)
    print('act_space:',act_space)

    #n_agents = len(args["agents"])
    grouping = {'group1':env.possible_agents}
    return env.with_agent_groups( grouping,obs_space=obs_space, act_space=act_space)

ray.shutdown()
ray.init(num_gpus=1)

register_env("grouped_test", env_creator)
"""results = tune.run(
    "QMIX",
    stop={"timesteps_total": 5,},
    config={
        "mixer": "qmix",
        "env": "grouped_test",
        "observation_space": {
        "attaccante": Box(low=0,high=1,shape=(2,),dtype=np.float32),
        "difensore": Box(low=0,high=1,shape=(2,),dtype=np.float32),
    },
        "env_config": {"agents": ["attaccante", "difensore"]},
        "num_workers": 0,
        "timesteps_per_iteration": 2,
        "observetion_space":None,
    },
) """

tf1, tf, tfv = try_import_tf()
torch, _ = try_import_torch()


2023-09-26 17:24:29,948	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [59]:
config = MADDPGConfig().environment("grouped_test").framework("torch").resources(num_gpus=1)
#config = PGConfig().environment("grouped_test").framework("torch").resources(num_gpus=1)
stop = {
        "training_iteration": 150,
        "timesteps_total": 100000,
        "episode_reward_mean": 1000.0,
    }
""" results = tune.Tuner(
        "PG", param_space=config, run_config=air.RunConfig(stop=stop, verbose=1)
    ).fit() """
results = tune.Tuner(
        "MADDPG", param_space=config, run_config=air.RunConfig(stop=stop, verbose=1)
    ).fit()
print(results)

(pid=461673) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(pid=461673) 2023-09-26 17:25:35,895	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
(pid=461673) 2023-09-26 17:25:35,900	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
(MADDPG pid=461673) 2023-09-26 17:25:35,972	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/maddpg/` has been deprecated. Use `rllib_contrib/maddpg/` instead. This will raise an error in the future!
(MADDPG pid=461673) 2023-09-26 17:25:35,972	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/simple_q/` has been deprecated. Use `rllib_contrib/simple_q/` instead. This will raise an error in the future!
(MADDPG pid=461673) 2023-09-26 17:25:35,973	WARNING algorithm_config.py:672 -- Cannot create MADDPGConfig from 

(RolloutWorker pid=461723) obs_space: {'attaccante': Box(0.0, 1.0, (2,), float32), 'difensore': Box(0.0, 1.0, (2,), float32)}
(RolloutWorker pid=461723) act_space: {'attaccante': Discrete(3), 'difensore': Discrete(3)}
(RolloutWorker pid=461723) Agente in azione: attaccante
(RolloutWorker pid=461723) Agente in azione: difensore


2023-09-26 17:25:47,615	ERROR tune.py:1139 -- Trials did not complete: [MADDPG_grouped_test_e9b2a_00000]
2023-09-26 17:25:47,616	INFO tune.py:1143 -- Total run time: 23.48 seconds (23.45 seconds for the tuning loop).
2023-09-26 17:25:47,619	WARNING experiment_analysis.py:205 -- Failed to fetch metrics for 1 trial(s):
- MADDPG_grouped_test_e9b2a_00000: FileNotFoundError('Could not fetch metrics for MADDPG_grouped_test_e9b2a_00000: both result.json and progress.csv were not found at /home/matteo/ray_results/MADDPG_2023-09-26_17-25-24/MADDPG_grouped_test_e9b2a_00000_0_2023-09-26_17-25-24')


ResultGrid<[
  Result(
    error='TuneError',
    metrics={},
    path='/home/matteo/ray_results/MADDPG_2023-09-26_17-25-24/MADDPG_grouped_test_e9b2a_00000_0_2023-09-26_17-25-24',
    filesystem='local',
    checkpoint=None
  )
]>


In [60]:
def select_policy(agent_id, episode, **kwargs):
        if agent_id == "attaccante":
            return "learned"
        else:
            return random.choice(["always_same", "beat_last"])

config = (
        PGConfig()
        .environment("grouped_test")
        .framework("torch")
        .rollouts(
            num_rollout_workers=0,
            num_envs_per_worker=4,
            rollout_fragment_length=10,
        )
        .training(
            train_batch_size=200,
            gamma=0.9,
        )
        .multi_agent(
            policies={
                "always_same": PolicySpec(policy_class=AlwaysSameHeuristic),
                "beat_last": PolicySpec(policy_class=BeatLastHeuristic),
                "learned": PolicySpec(
                    config=AlgorithmConfig.overrides(
                        model={"use_lstm": False},
                        framework_str="torch",
                    )
                ),
            },
            policy_mapping_fn=select_policy,
            policies_to_train=["learned"],
        )
        .reporting(metrics_num_episodes_for_smoothing=200)
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )
algo = config.build()

/home/matteo/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/matteo/.local/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/matteo/.local/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray

obs_space: {'attaccante': Box(0.0, 1.0, (2,), float32), 'difensore': Box(0.0, 1.0, (2,), float32)}
act_space: {'attaccante': Discrete(3), 'difensore': Discrete(3)}
Agente in azione: attaccante
Agente in azione: difensore


ValueError: `observation_space` not provided in PolicySpec for always_same and env does not have an observation space OR no spaces received from other workers' env(s) OR no `observation_space` specified in config!